In [ ]:
from netCDF4 import Dataset
import numpy as np
from metpy.plots import wx_code_to_numeric
import pandas as pd

import numpy as np

import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import current_weather, sky_cover, StationPlot, simple_layout
from metpy.io import metar

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
data = Dataset('.20190527_1900')

In [ ]:
#Read in variables needed for plotting surface station plots

station_name = data.variables['stationName'[:]
latitude = data.variables['latitude'][:]
longitude = data.variables['longitude'][:]
temp = data.variables['temperature'][:]
dwpt = data.variables['dewpoint'][:]
wspd = data.variables['windSpeed'][:]
wdir = data.variables['windDir'][:]
gust = data.variables['windGust'][:]
weather = data.variables['presWeather'][:]
visibility = data.variables['visibility'][:]
pslp = data.variables['seaLevelPress'][:]
skycover = data.variables['skyCover'][:]

#if getting an error, change data to nc!



In [ ]:
#Convert station names from masked byte array to list of strings

stn_names = ['']*len(station_name)]

for index in range(len(station_name)):
    stn = [station_name[index].tobytes().decode().rstrip('\x00')
    stn_names[index] = stn       

#Convert present weather from a masked byte array to a list of strings
cur_wx = ['']*len(weather)
for index in range(len(weather)):
    wx = weather[index].tobytes().decode().rstrip('\x00')
    cur_wx[index] = wx

#Converting the current weather string to numerical value for metpy plotting
wx_codes = wx_code_to_numeric(cur_wx)


sky_cover_num = np.empty(len(skycover))
for i in range(len(skycover)):
    this_skycvr = skycover[i].tobytes().decode().rstrip('\x00')
    if this_skycvr[-3:] == 'SKC' or this_skycvr[-3:] == 'CLR':
        sky_cover_num[i] = 0
    elif this_skycvr[-3:] == 'FEW':
        sky_cover_num[i] = 2
    elif this_skycvr[-3:] == 'SCT':
        sky_cover_num[i] = 4
    elif this_skycvr[-3:] == 'BKN':
        sky_cover_num[i] = 6
    elif this_skycvr[-3:] == 'OVC':
        sky_cover_num[i] = 8

    else:
        sky_cover_num[i] = np.nan
        
print(sky_cover[-3:])

#if skc = 0/8
#if few = 2/8
#if scattered (sct) = 4/8
#if broken (bkn) = 6/8
#if overcast(ovc) = 8/8


#How could we do this for all the elements? Put it in a loop!

#netCDF files do not like using strings!!! Use an empty string [''] for stn_names instead of numpy!!



In [ ]:
#Get u and v wind components

wspd_from_nc = np.array(wspd)*units('m/s')
wdir_from_nc = np.array(wdir)*units('m/s')
u,v = mpcalc.wind_components(wspd_from_nc, wdir_from_nc)

#new terminal
#conda activate atsc270
#cd ATSC_270
#go to project directory
#wdir is in degrees
#wind speed in m/s

#visibility is in meters, but convert to miles (*units('meters').to('miles'))
#slp in pa, so convert to hpa (divide by 100)


In [ ]:
#Convert our data to a pandas dataframe
data_for_df = [stn_names,latitude,longitude,wdir,wspd,gust,visibility,wx_codes,temp,dwpt,pslp]
col_names = ['station_id':stn_names,
             'latitude':latitude, 
             'longitude':longitude, 
             'wind_direction':wdir, 
             'wind_speed':wspd, 
             'wind_gust':gust, 
             'visibility':visibility, 
             'current_wx1_symbol':wx_codes,
             'air_temperature':temp, 
             'dew_point_temperature':dwpt, 
             'air_pressure_at_sea_level':pslp
             'eastward_wind':u,
             'northward_wind':v,
             'cloud_cov': skyCover]

                
df = pd.DataFrame(data=data_for_df, index=stn_names) #columns=col_names) ? 
#May have to remove index=stn_names, so that station name/ID is not in bracket
print(df)